# Agentic AI MCP - Client Only

This notebook connects to a remote MCP server and runs agentic workflows.
The MCP server should be running on another machine (see `server.ipynb`).


## Step 1: Setup

In [1]:
# install if needed
# !uv pip install agentic-ai-mcp==0.5.0

import agentic_ai_mcp
print(f"Version: {agentic_ai_mcp.__version__}")

Version: 0.5.0


## Step 2: Make sure you have a valid API_KEY

The LLM runs on this machine, so you need the API key here.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.getenv("ANTHROPIC_API_KEY"):
    print("ANTHROPIC_API_KEY is set")
else:
    print("WARNING: ANTHROPIC_API_KEY not found. Set it in your .env file.")

ANTHROPIC_API_KEY is set


## Step 3: Connect to Remote MCP Server

Use the `mcp_url` parameter to connect to an existing MCP server.

In [3]:
from agentic_ai_mcp import AgenticAI

# connect to remote MCP server (change this to your server's URL)
MCP_SERVER_URL = "http://127.0.0.1:8888/mcp"  # change to remote IP if needed

ai = AgenticAI(mcp_url=MCP_SERVER_URL, verbose=True)  # client-only mode: connect to existing server

print(f"Client configured to connect to: {MCP_SERVER_URL}")

Client configured to connect to: http://127.0.0.1:8888/mcp


## Step 4a: Run Agent (Simple Task)

In [8]:
result = await ai.run("calculate 2 + 2, then greet 'tom' the result number of times")
print()
print('>>>')
print()
print(result)


PROMPT: calculate 2 + 2, then greet 'tom' the result number of times

STEP 1: add({'a': 2, 'b': 2})
  → CallToolResult(content=[TextContent(type='text', text='4', annotations=None, meta=None)], structured_content={'result': 4}, meta=None, data=4, is_error=False)

STEP 2: greet({'name': 'tom', 'times': 4})
  → CallToolResult(content=[TextContent(type='text', text='Hello, tom! Hello, tom! Hello, tom! Hello, tom! ', annotations=None, meta=None)], structured_content={'result': 'Hello, tom! Hello, tom! Hello, tom! Hello, tom! '}, meta=None, data='Hello, tom! Hello, tom! Hello, tom! Hello, tom! ', is_error=False)

RESULT: Perfect! 2 + 2 = 4, and I greeted tom 4 times: "Hello, tom! Hello, tom! Hello, tom! Hello, tom!"


>>>

Perfect! 2 + 2 = 4, and I greeted tom 4 times: "Hello, tom! Hello, tom! Hello, tom! Hello, tom!"


## Step 4b: Run Agent (Complex Task)

In [11]:
result = await ai.run_with_planning("calculate (((0+1)+(1+0))+1), then greet 'Alice' the result number of times")
print()
print('>>>')
print()
print(result)


PLANNING MODE
TASK: calculate (((0+1)+(1+0))+1), then greet 'Alice' the result number of times

PLAN:
  1. Use add(a=0, b=1) to calculate the first inner sum
  2. Use add(a=1, b=0) to calculate the second inner sum
  3. Use add(a=result from step 1, b=result from step 2) to add those results
  4. Use add(a=result from step 3, b=1) to add 1 to get the final value
  5. Use greet(name="Alice", times=result from step 4) to greet Alice that many times

EXECUTING STEP 1/5: Use add(a=0, b=1) to calculate the first inner sum
----------------------------------------
  TOOL CALL 1: add({'a': 0, 'b': 1})
    → CallToolResult(content=[TextContent(type='text', text='1', annotations=None, meta=None)], structured_content={'result': 1}, meta=None, data=1, is_error=False)
  AI RESPONSE: The first inner sum is **1** (0 + 1 = 1).
----------------------------------------
STEP 1 COMPLETE

EXECUTING STEP 2/5: Use add(a=1, b=0) to calculate the second inner sum
----------------------------------------
  TOO